In [1]:
import re
from aocd import get_data, submit 
from dataclasses import dataclass

# @dataclass(frozen=True)
# class Money:
#     ore: int = 0
#     clay: int = 0
#     obsidian: int = 0
#     geode: int = 0
        
#     def __add__(self, other):
#         return Money(
#             self.ore + other.ore,
#             self.clay + other.clay,
#             self.obsidian + other.obsidian,
#             self.geode + other.geode)
    
#     def __mul__(self, other):
#         return Money(
#             self.ore * other,
#             self.clay * other,
#             self.obsidian * other,
#             self.geode * other)
    
#     def __rmul__(self, other):
#         return self.__mul__(other)
    
#     def __gt__(self, other):
#         return (
#             (self.ore > other) and
#             (self.clay > other) and
#             (self.obsidian > other) and
#             (self.geode > other) )

# @dataclass(frozen=True)
# class Blueprint:
#     ore_robot: Money
#     clay_robot: Money
#     obsidian_robot: Money
#     geode_robot: Money

# @dataclass(frozen=True)
# class State:
#     money: Money = Money()
#     n_ore_bots: int = 0
#     n_clay_bots: int = 0
#     n_obsidian_bots: int = 0
#     n_geode_bots: int = 0
        
# def produce(s: State) -> State:
#         m = s.money + Money(
#             s.n_ore_bots, 
#             s.n_clay_bots, 
#             s.n_obsidian_bots, 
#             s.n_geode_bots)
#         return State(m, s.n_ore_bots, s.n_clay_bots, s.n_obsidian_bots, s.n_geode_bots)
    
# data = get_data(year=2022, day=19)
# data = """Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
# Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian."""
# data = data.split("\n")
# data = [list(map(int, re.findall(r"( [\d]* )", line))) for line in data]
# data = [Blueprint(
#     Money(ore=d[0]),
#     Money(ore=d[1]),
#     Money(ore=d[2], clay=d[3]),
#     Money(ore=d[4], obsidian=d[5])
#         ) for d in data]

# data

In [76]:
import numpy as np
data = get_data(year=2022, day=19)
data = """Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian."""
data = data.split("\n")
data = [list(map(int, re.findall(r"( [\d]* )", line))) for line in data]
data = [(
        (d[0], 0, 0, 0),
        (d[1], 0, 0, 0),
        (d[2], d[3], 0, 0),
        (d[4], 0, d[5], 0)) for d in data]

bp = data[1]
bp


((2, 0, 0, 0), (3, 0, 0, 0), (3, 8, 0, 0), (3, 0, 12, 0))

In [77]:
# 10x speed improvement over numpy approach 
def buy_options(bp, m):
    return ((m[0] - bp[0][0] >= 0),
            (m[0] - bp[1][0] >= 0),
            (m[0] - bp[2][0] >= 0) and (m[1] - bp[2][1] >= 0),
            (m[0] - bp[3][0] >= 0) and (m[2] - bp[3][2] >= 0))

# 30x speed improvement over np.all(x >= 0)
all_nonnegative = lambda x: (x[0] >= 0) and (x[1] >= 0) and (x[2] >= 0) and (x[3] >= 0)

In [78]:
produce = lambda b, m: (m[0]+b[0], m[1]+b[1], m[2]+b[2], m[3]+b[3])
add_bot = lambda bots, idx: tuple((b+1 if idx==i else b) for i, b in enumerate(bots))

In [79]:
buy_bot = lambda bp, m, idx: (m[0]-bp[idx][0], m[1]-bp[idx][1], m[2]-bp[idx][2], m[3]-bp[idx][3])

In [80]:
buy_options(bp, (5,5,5,0))

(True, True, False, False)

In [92]:
is_better_money = lambda a, b: False if a == b else ((a[0] >= b[0]) and (a[1] >= b[1]) 
                                                        and (a[2] >= b[2]) and (a[3] >= b[3]))

In [82]:
is_better_money((8,4,0,0),(8,4,0,0))

False

In [96]:
is_dumb_state_heuristic = lambda b: (b[0] > 10) or (b[1] > 10)

In [ ]:
is_not_productive_heuristic = lambda i, m: (i >= 20) and (m[3] == 0)

In [97]:



def find_max_geodes(bp):
    states = set([
        ((1, 0, 0, 0), (0, 0, 0, 0), (0, 0, 0, 0))
    ])
    new_best_money_per_bots = {(1,0,0,0): (-1,-1,-1,-1)}
    for i in range(24):
        best_money_per_bots = new_best_money_per_bots.copy()
        new_states = set()
        for state in states: 
            bots, money, prev_buy_options = state
            if is_better_money(best_money_per_bots[bots], money):
                continue
            if is_dumb_state_heuristic(bots):
                continue
            if is_not_productive_heuristic(i, money)
            options = buy_options(bp, money)
            money = produce(bots, money)
            new_money = [money]
            new_bots = [bots]
            new_prev_buy_options = [options]
            if not options == prev_buy_options:
                for idx, option in enumerate(options):
                    if option: 
                        bots_option = add_bot(bots, idx)
                        money_option = buy_bot(bp, money, idx)
                        if bots_option in new_best_money_per_bots:
                            if is_better_money(money_option, new_best_money_per_bots[bots_option]):
                                new_best_money_per_bots[bots_option] = money_option
                        else:
                            new_best_money_per_bots[bots_option] = money_option
                        new_money.append(money_option)
                        new_bots.append(bots_option)
                        new_prev_buy_options.append((-1, -1, -1, -1))
            new_states.update([(b,m, opt) for b, m, opt in zip(new_bots, new_money, new_prev_buy_options)])
        states = new_states
        print(f"{i}: {len(states)} state{'s' if len(states) > 1 else ''}")

    geodes = [s[1][3] for s in states]
    return states, max(geodes)
states, g = find_max_geodes(bp)
g

0: 1 state
1: 1 state
2: 2 states
3: 4 states
4: 7 states
5: 12 states
6: 25 states
7: 42 states
8: 71 states
9: 134 states
10: 295 states
11: 550 states
12: 1316 states
13: 3050 states
14: 7135 states
15: 16381 states
16: 35303 states
17: 75667 states
18: 165942 states
19: 347114 states
20: 781756 states
21: 1913846 states
22: 4775264 states
23: 11832666 states


12

In [85]:
states = list(states)

In [89]:
len([(bots, money) for bots, money, _ in states if bots[0] <= 5])

2716119

In [90]:
[(bots, money) for bots, money, _ in states if money[3] == 12]

[((4, 6, 8, 3), (15, 29, 22, 12)),
 ((5, 7, 6, 3), (19, 45, 18, 12)),
 ((4, 7, 6, 3), (13, 44, 19, 12)),
 ((3, 6, 7, 3), (12, 25, 21, 12)),
 ((5, 6, 6, 3), (21, 44, 18, 12)),
 ((4, 6, 7, 3), (22, 25, 18, 12)),
 ((6, 6, 6, 3), (20, 45, 18, 12)),
 ((4, 6, 8, 3), (15, 28, 23, 12)),
 ((3, 7, 6, 3), (12, 47, 19, 12)),
 ((4, 7, 7, 3), (8, 40, 18, 12)),
 ((7, 5, 6, 3), (32, 32, 18, 12)),
 ((4, 6, 7, 4), (16, 36, 9, 12)),
 ((4, 7, 6, 3), (10, 47, 19, 12)),
 ((5, 6, 6, 3), (12, 45, 18, 12)),
 ((4, 7, 7, 3), (16, 37, 21, 12)),
 ((4, 7, 6, 3), (11, 52, 19, 12)),
 ((4, 6, 6, 3), (22, 44, 18, 12)),
 ((5, 6, 6, 3), (32, 32, 18, 12)),
 ((4, 6, 6, 3), (13, 44, 19, 12)),
 ((5, 7, 6, 3), (29, 36, 18, 12)),
 ((4, 6, 7, 4), (10, 37, 7, 12)),
 ((4, 7, 7, 3), (8, 44, 19, 12)),
 ((4, 8, 6, 4), (12, 49, 6, 12)),
 ((6, 6, 6, 3), (22, 35, 18, 12)),
 ((5, 6, 6, 4), (17, 45, 6, 12)),
 ((5, 6, 6, 4), (17, 44, 7, 12)),
 ((5, 8, 6, 3), (6, 55, 19, 12)),
 ((4, 7, 6, 3), (10, 48, 18, 12)),
 ((4, 5, 7, 4), (22, 24, 9, 

In [91]:
len([(bots, money) for bots, money, _ in states if bots[0] > 10])

1340570

In [10]:
from datetime import datetime
print(datetime.now())
import cProfile

with cProfile.Profile() as pr:
    find_max_geodes(data[1])
    pr.print_stats()


2022-12-20 19:05:17.968370


In [91]:
[s for s in states if s[1] == (6, 41, 8, 9)]

[((1, 4, 2, 2), (6, 41, 8, 9))]

In [3]:
import numpy as np 
x = np.array([0,1,2,3])

In [4]:
%timeit np.all(x > 0)

4.87 µs ± 107 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [6]:
%timeit all_positive(x)

178 ns ± 0.784 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [ ]:
# from itertools import groupby
# from collections import defaultdict

# # def reduce_states(states):
# #     states = sorted(list(states))
# #     output = set()
# #     for bots, m_gen in groupby(states, lambda s: s[0]):
# #         moneys = [m[1] for m in m_gen]
# #         best_moneys = find_best_moneys(moneys)
# # #         print(bots, moneys)
# # #         print(bots, best_moneys)
# #         for bmoney in best_moneys:
# #             output.add((bots, bmoney))
# #     return output

# def reduce_states(states):
#     states_dict = defaultdict(list)
#     for bots, moneys in states:
#         states_dict[bots].append(moneys)
#     output = set()
#     for bots, moneys in states_dict.items():
#         best_moneys = find_best_moneys(moneys)
#         for bmoney in best_moneys:
#             output.add((bots, bmoney))
#     return output

# def find_best_moneys(moneys):
#     moneys.sort(reverse=True)
#     best = set()
#     new_set = set()
#     for item in moneys:
#         item = np.array(item)
#         best = new_set
#         new_set = best.copy()
#         for b in best:
#             b = np.array(b)
#             if np.all(b - item >= 0):
#                 break
#         else:
#             new_set.add(tuple(item))
#             for b in best:
#                 b = np.array(b)
#                 if(np.all(item - b >= 0)):
#                     new_set.remove(tuple(b))
#         best = new_set
#     return new_set
            

In [59]:
x = (0,1,2)
a = x 
a[0] = 5
print(x)
print(a)

TypeError: 'tuple' object does not support item assignment

In [18]:
(False, True, False, False) != (False, False, False, False)

True